## In this notebook, amplicon data of two different marker genes from the same samples will be imported as raw read counts. After modeling ASV occurrences as probability distributions and extracting principal components of both data sets, a Bayesian model will be fit to estimate the parameters of a normal likelihood.

### Import all the dependencies

In [1]:
import os
import pandas as pd
import numpy as np
import subprocess

### Move to working directory

In [2]:
os.chdir("/Users/nastassia.patin/GitHub/NOAA-NCAR-Hackathon/Data")

## 01. This section imports the amplicon data sets as raw counts and calls an R script to model the ASV occurrences as probability distributions. 

#### Import amplicon data sheets as pandas dataframes and take a look

In [3]:
file1 = 'Flyer2018_16S_table_counts.tsv'
file2 = 'Flyer2018_18S_table_counts.tsv'
asvs1 = pd.read_csv(file1, index_col=0, sep='\t')
asvs2 = pd.read_csv(file2, index_col=0, sep='\t')

In [4]:
asvs1

,CN18Fc12_8_eDNA,CN18Fc19_5_eDNA,CN18Fc21_6_eDNA,CN18Fc22_6_eDNA,CN18Fc24_6_eDNA,CN18Fc25_5_eDNA,CN18Fc27_4_eDNA,CN18Fc29_6_eDNA,CN18Fc30_4_eDNA,CN18Fc32_4_eDNA,...,CN18SESPkoa_SC36,CN18SESPkoa_SC37,CN18SESPkoa_SC39,CN18SESPkoa_SC40,CN18SESPkoa_SC41,CN18SESPkoa_SC42,CN18SESPkoa_SC44,CN18SESPkoa_SC45,CN18SESPkoa_SC47,CN18SESPkoa_SC49
ASV ID,,,,,,,,,,,,,,,,,,,,,
495c1bd1608a1dad54d3e2824ce899ef,552,7415,8749,8152,7124,12422,215,8080,8799,2231,...,4324,1339,95,1554,147,2720,21,2015,1847,1886
a900b6678ce86851fb16bfafb87f3326,210,1933,2808,1967,1671,4912,57,3688,3343,1134,...,21023,8898,1367,8291,1112,9881,60,8726,9330,8852
c8e360969108fa2125a3d56eb4dad24f,145,2089,2530,2086,2343,2395,129,1625,1664,639,...,4830,9,566,20,551,60,43,22,28,44
72143fd9e63fe40c1258948d2f0d79c3,130,1830,2516,2178,2256,3332,56,2535,2316,629,...,4227,83,512,75,497,154,48,97,116,80
7b6b178fad5599c0e9a734e4fb09fd64,156,1742,1761,1855,1812,2467,73,1663,1612,515,...,2385,47,287,45,520,104,50,57,72,49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
674933a0d44342a0647f7a5b4591f26e,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
bebe1b9a7e9aaa78172c1208111f4570,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0128431733f67d02efad766d717fe6fd,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
asvs2

,CN18Fc12_8_eDNA,CN18Fc19_5_eDNA,CN18Fc21_6_eDNA,CN18Fc22_6_eDNA,CN18Fc24_6_eDNA,CN18Fc25_5_eDNA,CN18Fc27_4_eDNA,CN18Fc29_6_eDNA,CN18Fc30_4_eDNA,CN18Fc32_4_eDNA,...,CN18SESPkoa_SC36,CN18SESPkoa_SC37,CN18SESPkoa_SC39,CN18SESPkoa_SC40,CN18SESPkoa_SC41,CN18SESPkoa_SC42,CN18SESPkoa_SC44,CN18SESPkoa_SC45,CN18SESPkoa_SC47,CN18SESPkoa_SC49
ASV ID,,,,,,,,,,,,,,,,,,,,,
ASV_1,1,0,3,0,0,3,0,1,7,2,...,7,2,3,2,1,2,1,2,5,4
ASV_2,0,0,0,0,0,0,0,0,1,0,...,2,0,0,0,2,1,2,1,1,1
ASV_3,2,0,23,6,5,0,0,3,1,0,...,1,86,2,51,2,94,2,12,24,13
ASV_4,7,13521,3,2215,23301,16490,9,5,19,3199,...,4,0,1,1,0,1,2,0,1,3
ASV_5,11,13,31,40,23,34,37,19,42,15,...,3001,15,2707,32,610,53,1833,46,64,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ASV_12614,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ASV_12622,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ASV_12628,0,0,0,0,0,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0


### Call the R script to model the ASV count distributions

#### Function to run R program on input data given the file name

In [13]:
def call_Rscript_for_amplicon_modeling(path, sample_num, arg1):
    # Use subprocess.Popen to open executable Rscript and extract stdout from 'print' command in R
    with subprocess.Popen(['Rscript', path, '--args', '--vanilla', arg1], 
                          stdout=subprocess.PIPE) as result:
        asvs_freq = result.stdout.read()
    
    # The R output gets imported as "bytes"; need to convert to string and remove whitespace
    y = asvs_freq.decode("utf-8")
    y = y.split()

    # Convert to numpy array
    array = np.asarray(y)
    # Reshape to original dimensions; number of samples (columns) is second field, 
    # unknown number of ASVs can be supplied with '-1'
    mat = np.reshape(array, (-1, sample_num))

    # Convert to data frame with float values instead of strings
    df = pd.DataFrame(mat)
    df = df.astype(float)
    return(df)

#### Define the arguments (Rscript path)

In [14]:
path_to_rscript = '/Users/nastassia.patin/GitHub/NOAA-NCAR-Hackathon/ranRelPct_testdata.R'

#### Run the function in a loop over both amplicon data sets and make a list of two data frames

In [15]:
df = []

for file in [file1, file2]:
    asvs = pd.read_csv(file, index_col=0, sep='\t')
    number_of_samples = len(asvs.axes[1])
    asvs_modeled = call_Rscript_for_amplicon_modeling(path_to_rscript,
                                      number_of_samples,
                                      file)
    df.append(asvs_modeled)

#### Separate the two data frames into the 16S and 18S modeled ASV counts and transpose for PCA

In [9]:
df_16S = df[0].T
df_18S = df[1].T

In [18]:
df[0].T

,0,1,2,3,4,5,6,7,8,9,...,2742,2743,2744,2745,2746,2747,2748,2749,2750,2751
0,0.081235,0.031577,0.021459,0.020138,0.019784,0.006785,0.000231,0.012197,0.030748,0.012443,...,4.616968e-05,5.663632e-05,0.000031,0.000449,0.000128,0.000057,0.000096,0.000299,0.000140,4.212637e-05
1,0.169258,0.044937,0.045780,0.041952,0.040051,0.011598,0.006134,0.012862,0.016433,0.020236,...,4.749789e-05,4.134312e-05,0.000027,0.000020,0.000025,0.000011,0.000022,0.000032,0.000009,1.524954e-05
2,0.132444,0.042060,0.039251,0.038707,0.026791,0.011675,0.009388,0.018799,0.023767,0.019280,...,5.838843e-07,3.070425e-05,0.000008,0.000020,0.000007,0.000006,0.000004,0.000006,0.000003,5.534466e-06
3,0.152946,0.037359,0.038312,0.042988,0.034844,0.009951,0.007114,0.014646,0.018590,0.017366,...,1.119618e-05,4.370416e-05,0.000006,0.000025,0.000006,0.000038,0.000001,0.000016,0.000004,3.093478e-09
4,0.133530,0.031385,0.043268,0.040892,0.032659,0.013455,0.005570,0.016304,0.022474,0.014633,...,3.165752e-06,8.941196e-06,0.000003,0.000005,0.000041,0.000025,0.000002,0.000016,0.000006,2.023532e-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,0.046058,0.165288,0.001077,0.002948,0.001945,0.010987,0.045270,0.003258,0.001815,0.005349,...,4.501413e-06,4.385110e-07,0.000014,0.000012,0.000028,0.000043,0.000008,0.000035,0.000017,3.983290e-05
58,0.002917,0.008231,0.006781,0.006720,0.009478,0.039610,0.000057,0.030021,0.004003,0.000010,...,3.679679e-04,4.165121e-05,0.000121,0.000103,0.000109,0.000019,0.000125,0.000129,0.000153,2.559747e-04
59,0.038357,0.163379,0.000482,0.001812,0.001130,0.006987,0.053997,0.001677,0.001994,0.003067,...,5.886548e-06,1.347213e-05,0.000011,0.000013,0.000022,0.000010,0.000003,0.000050,0.000008,1.701321e-05
60,0.031143,0.153632,0.000500,0.001727,0.001102,0.004991,0.040818,0.001543,0.001167,0.003921,...,1.086868e-05,2.877440e-05,0.000043,0.000034,0.000007,0.000023,0.000044,0.000006,0.000003,7.399838e-06


#### Add column and row names and export as csv files

In [11]:
df_16S.columns = list(asvs1.index)
df_16S.index = list(asvs1.columns)

In [13]:
df_18S.columns = list(asvs2.index)
df_18S.index = list(asvs2.columns)

In [15]:
df_16S.to_csv("Flyer2018_16S_counts_modeled.tsv", sep='\t')
df_18S.to_csv("Flyer2018_18S_counts_modeled.tsv", sep='\t')

## 02. Pass modeled ASV counts tables to Bayesian modeling R Script

In [136]:
def bayesian_modeling_of_two_markergenes(path, filename_16S, filename_18S):
    # Use subprocess.Popen to open executable Rscript and extract stdout from 'print' command in R
    with subprocess.Popen(['Rscript', path, '--args', '--vanilla', filename_16S,
                      filename_18S], stdout=subprocess.PIPE) as result:
        bayes_summary = result.stdout.read()
    # Convert bytes to string
    x = bayes_summary.decode("utf-8")
    # Remove newline symbol
    y = x.replace("\n", "" ).split()
    z = [ x for x in y if "\"" in x ]
    # Convert to numpy array
    array = np.asarray(z)
    # Reshape array to dimensions of post.summary matrix
    mat = np.reshape(array, (-1, 12))

    # Convert to data frame
    df = pd.DataFrame(mat)
    # Remove all the " from string values
    for i, col in enumerate(df.columns):
        df.iloc[:, i] = df.iloc[:, i].str.replace('"', '')
    # Convert first row to column names and first column to row names (index)
    df.columns = df.loc[0, :]
    df = df[1:]
    df = df.set_index(df.columns[0])
    # Convert strings to floats
    df = df.astype(float)
    return(df)

#### Define variables

In [137]:
path_to_rescript = "/Users/nastassia.patin/GitHub/NOAA-NCAR-Hackathon/PC_bayesian_runner_testdata.R"
filename_16S = "Flyer2018_16S_counts_modeled.tsv"
filename_18S = "Flyer2018_18S_counts_modeled.tsv"

#### Run the function

In [138]:
bayes_summary_16S_18S = bayesian_modeling_of_two_markergenes(path_to_rscript, filename_16S, filename_18S)

#### Display the summary output

In [140]:
bayes_summary_16S_18S

,Lower95,Median,Upper95,Mean,SD,Mode,MCerr,MC%ofSD,SSeff,AC.100,psrf
rownames,,,,,,,,,,,
deviance,727.704000,749.967000,774.663000,750.948830,12.250951,748.554864,0.124009,1.0,9760.0,0.017334,1.000067
intercept[1],-1.310640,0.008855,1.313310,0.011588,0.672370,-0.006323,0.006767,1.0,9873.0,-0.011665,1.000148
intercept[2],-1.218190,0.005048,1.315150,0.005015,0.643597,-0.005985,0.006507,1.0,9784.0,0.003424,1.000248
"b[1,1]",-1.339490,-1.256965,-1.180580,-1.257152,0.040722,-1.256734,0.000415,1.0,9626.0,-0.001085,0.999876
"b[2,1]",-0.370138,-0.289576,-0.201671,-0.289082,0.042669,-0.289438,0.000427,1.0,10000.0,0.022967,1.000117
"b[3,1]",-0.024914,0.062410,0.141932,0.062005,0.042577,0.062734,0.000430,1.0,9811.0,-0.004888,1.000010
"b[4,1]",0.082799,0.169481,0.257953,0.169753,0.044298,0.165584,0.000454,1.0,9522.0,0.013740,1.000194
"b[5,1]",-0.145362,-0.052085,0.036401,-0.052971,0.046350,-0.051781,0.000475,1.0,9503.0,0.004122,1.000008
"b[6,1]",0.176274,0.267029,0.361548,0.267298,0.046990,0.261878,0.000473,1.0,9866.0,0.000919,1.000250
